In [1]:
import pandas as pd

pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 100
input_file = "4096x5120x4096.gemm.csv"
no_search_input = "./no_search/4096x5120x4096.gemm.csv"

In [ ]:
COLS_TO_DROP = [
    "Problem",
    "Provider",
    "OperationKind",
    "gemm_kind",
    "alpha",
    "beta",
    "enable_sm90_mixed_dtype_shuffle_test",
    "min_cc",
    "max_cc",
    "op_class",
    "Bytes",
    "Flops",
    "Runtime",
    "C",
]


def load_df(path: str, cols_to_drop=COLS_TO_DROP):
    df = pd.read_csv(path)
    df = df.copy()
    df = df.drop(
        labels=cols_to_drop,
        axis=1,
    )
    df.sort_values(by="GFLOPs", ascending=False, inplace=True)
    return df

Index(['Operation', 'Disposition', 'Status', 'm', 'n', 'k', 'A', 'B', 'D',
       'split_k_mode', 'split_k_slices', 'batch_count', 'raster_order',
       'runtime_input_datatype_a', 'runtime_input_datatype_b', 'use_pdl',
       'swizzle_size', 'accum', 'cta_m', 'cta_n', 'cta_k', 'cluster_m',
       'cluster_n', 'cluster_k', 'cluster_m_fallback', 'cluster_n_fallback',
       'cluster_k_fallback', 'stages', 'warps_m', 'warps_n', 'warps_k',
       'inst_m', 'inst_n', 'inst_k', 'Flops/Byte', 'GB/s', 'GFLOPs'],
      dtype='object')

In [13]:
df.head()

,Operation,Disposition,Status,m,n,k,A,B,D,split_k_mode,split_k_slices,batch_count,raster_order,runtime_input_datatype_a,runtime_input_datatype_b,use_pdl,swizzle_size,accum,cta_m,cta_n,cta_k,cluster_m,cluster_n,cluster_k,cluster_m_fallback,cluster_n_fallback,cluster_k_fallback,stages,warps_m,warps_n,warps_k,inst_m,inst_n,inst_k,Flops/Byte,GB/s,GFLOPs
512,cutlass3x_sm90_tensorop_gemm_f16_f16_f32_f16_f16_256x128x64_1x2x1_0_tnn_align8_warpspecialized_c...,passed,success,4096,5120,4096,f16:row,f16:column,f16:column,serial,1,1,along_m,invalid,invalid,False,1,f32,256,128,64,1,1,1,0,0,0,4,4,2,1,64,128,16,1463,545.687,857336.0
97,cutlass3x_sm90_tensorop_gemm_f16_f16_f32_f16_f16_128x256x64_1x2x1_0_tnn_align8_warpspecialized_c...,passed,success,4096,5120,4096,f16:row,f16:column,f16:column,serial,1,1,along_m,invalid,invalid,False,1,f32,128,256,64,1,1,1,0,0,0,4,4,2,1,64,256,16,1463,543.430,853791.0
121,cutlass3x_sm90_tensorop_gemm_f16_f16_f32_f16_f16_128x256x64_1x1x1_0_tnn_align8_warpspecialized_c...,passed,success,4096,5120,4096,f16:row,f16:column,f16:column,serial,1,1,along_m,invalid,invalid,False,2,f32,128,256,64,1,1,1,0,0,0,4,4,2,1,64,256,16,1463,541.685,851049.0
153,cutlass3x_sm90_tensorop_gemm_f16_f16_f32_f16_f16_128x256x64_2x1x1_0_tnn_align8_warpspecialized_c...,passed,success,4096,5120,4096,f16:row,f16:column,f16:column,serial,1,1,along_n,invalid,invalid,False,4,f32,128,256,64,1,1,1,0,0,0,4,4,2,1,64,256,16,1463,541.334,850498.0
528,cutlass3x_sm90_tensorop_gemm_f16_f16_f32_f16_f16_256x128x64_1x1x1_0_tnn_align8_warpspecialized_c...,passed,success,4096,5120,4096,f16:row,f16:column,f16:column,serial,1,1,along_m,invalid,invalid,False,4,f32,256,128,64,1,1,1,0,0,0,4,4,2,1,64,128,16,1463,537.583,844604.0


In [18]:
df.iloc[0].Operation

'cutlass3x_sm90_tensorop_gemm_f16_f16_f32_f16_f16_256x128x64_1x2x1_0_tnn_align8_warpspecialized_cooperative_epi_tma'

In [12]:
from triton.testing import do_bench
import torch

M, N, K = 4096, 5120, 4096
dtype = torch.float16

A = torch.randn(M, K, dtype=dtype, device="cuda")
B = torch.randn(K, N, dtype=dtype, device="cuda")

GFLOP = (2 * M * N * K) / 1e9
time_ms = do_bench(lambda: torch.matmul(A, B))
time_s = time_ms / 1e3

print(f"Torch GFlops: {GFLOP / time_s:.2f}")


Torch GFlops: 754080.32
